In [26]:
!pip install pulp


In [27]:
!pip install ortools

In [28]:
import pandas as pd
from pulp import LpProblem, LpVariable, LpBinary, lpSum, LpStatus
from ortools.sat.python import cp_model

VERSION FUERZA BRUTA (COMO CLARIFICADOR)

In [ ]:
#VERSION FUERZA BRUTA

#from itertools import combinations
import pandas as pd

# Lista de facturas y sus importes como texto
raw_data = [
("V1250504920", "19.325,84 €"),
("V1250504923", "38.651,70 €"),
("V1250504919", "55.973,52 €"),
("V1250504922", "111.947,05 €"),
("V1250504921", "124.927,51 €"),
("V1250504924", "249.854,99 €"),
("V1250602662", "193,45 €"),
("V1250602663", "193,45 €"),
("V1250602664", "193,45 €"),
("V1250602665", "193,45 €"),
("V1250602661", "740,46 €"),
("V1250516164", "28.522,98 €"),
("V1250515208", "136.499,79 €"),
("V1250515205", "139.487,92 €"),
("V1250504926", "139.487,92 €"),
("V1250504932", "136.499,79 €"),
("V1250504928", "121.281,77 €"),
("V1250504925", "69.743,96 €"),
("V1250504927", "69.743,95 €"),
("V1250504930", "68.249,89 €"),
("V1250504933", "33.873,84 €"),
("V1250504929", "28.251,00 €"),
("V1250504931", "17.068,57 €"),
("V1250315385", "326,47 €"),
("V1250315386", "326,47 €"),
("V1250315387", "326,47 €"),
("V1250315388", "326,47 €"),
("V1250314423", "311,19 €"),
("V1250314419", "311,16 €"),
("V1250314420", "311,16 €"),
("V1250314421", "311,16 €"),
("V1250314422", "311,16 €"),

]

# Convertir los importes al formato numérico
data = [(id, float(imp.replace('.', '').replace(',', '.').replace(' €', ''))) for id, imp in raw_data]

# Objetivo a encontrar
target = 1284881.30

# Buscar combinaciones
for r in range(2, 12):  # Combos de 2 a 11 elementos (ajustable)
    for combo in combinations(data, r):
        total = sum(item[1] for item in combo)
        if abs(total - target) < 0.01:
            print("FACTURAS QUE SUMAN 1.284.881,30 €:")
            for factura, importe in combo:
                print(f"{factura} → {importe:.2f} €")
            print(f"\nTOTAL: {total:.2f} €")
            exit()
print("No se encontró una combinación exacta.")


No se encontró una combinación exacta.


VERSION CON PULP, DE 30 A 100 FACTURAS

In [ ]:
#VERSION CON PULP, DE 30 A 100 FACTURAS

# === DATOS (tu lista copiada directamente) ===
raw_data = [
   ("60GEC3700004", "133.727,53 €"),
("60EEC3000022", "125.165,62 €"),
("60GEC3000039", "125.165,62 €"),
("60FEC3000039", "125.165,62 €"),
("60EEC3000023", "125.165,62 €"),
("60EEC3000039", "61.636,53 €"),
("60GEC3000040", "61.636,53 €"),
("60FEC3000040", "61.636,53 €"),
("60EEC3000047", "61.636,53 €"),
("60EEC3000025", "35.710,05 €"),
("60EEC3000026", "35.710,05 €"),
("60EEC3000046", "35.710,05 €"),
("60EEC3000048", "35.710,05 €"),
("60GEC3000041", "35.710,05 €"),
("60GEC3000042", "35.710,05 €"),
("60FEC3000041", "35.710,05 €"),
("60FEC3000042", "35.710,05 €"),
("28FEC3000027", "34.790,89 €"),
("60EEC3000045", "34.568,64 €"),
("60GEC3000043", "34.568,64 €"),
("60FEC3000043", "34.568,64 €"),
("60EECR000001", "34.405,58 €"),
("60EEC3000024", "34.405,58 €"),
("35FEC3700001", "7.905,97 €"),
("35GEC3700002", "7.905,97 €"),
("60EEC3700004", "6.200,31 €"),
("60GEC3700003", "6.200,31 €"),
("60EEC3700005", "6.200,31 €"),
("60FEC3700005", "6.200,31 €"),
("28FEC3000028", "4.470,62 €"),
("28GEC3000044", "4.470,62 €"),
("28GEC3000046", "4.470,62 €"),
("28GEC3000047", "4.470,62 €"),
("28GEC3000045", "4.470,62 €"),
("28FEC3000029", "4.470,62 €"),
("28FEC3000025", "4.470,62 €"),
("60GEC3600004", "2.760,16 €"),
("60GEC3500004", "2.063,36 €"),
("TD-D5TG-105693", "1.113,90 €"),
("TD-E5TG-105643", "1.113,90 €"),
("TD-F5TG-105596", "1.113,90 €"),
("TD-G5TG-105581", "1.113,90 €"),
("28GEC3000042", "620,00 €"),
("60EEC3500003", "616,76 €"),
("60EEC3600003", "616,76 €"),
("60EEC3000044", "326,12 €"),
("28FEC3000026", "219,03 €"),
("60EEC3500004", "154,19 €"),
("60GEC3600003", "154,19 €"),
("60EEC3600004", "154,19 €"),
("60FEC3600003", "154,19 €"),
("60FEC3500003", "154,19 €"),
("60GEC3500003", "154,19 €")
]

# Convertir a formato numérico
data = [(id, float(imp.replace('.', '').replace(',', '.').replace(' €', ''))) for id, imp in raw_data]

# Objetivo (puedes cambiarlo fácilmente)
target = 259206.63

# Crear modelo de optimización
model = LpProblem("Seleccion_de_facturas", sense=1)

# Crear variables binarias (0 o 1) por cada factura
vars = {f_id: LpVariable(f"incluye_{f_id}", cat=LpBinary) for f_id, _ in data}

# Restricción: suma de importes seleccionados == objetivo
model += lpSum(vars[f_id] * importe for f_id, importe in data) == target

# Resolver
status = model.solve()

# Mostrar resultados
if LpStatus[status] == "Optimal":
    print("✅ Combinación encontrada:")
    total = 0
    for f_id, importe in data:
        if vars[f_id].value() == 1:
            print(f"{f_id} → {importe:.2f} €")
            total += importe
    print(f"\nTOTAL: {total:.2f} €")
else:
    print("❌ No se encontró una combinación exacta.")


✅ Combinación encontrada:
60GEC3700004 → 133727.53 €
60EEC3000046 → 35710.05 €
60FEC3000041 → 35710.05 €
35FEC3700001 → 7905.97 €
35GEC3700002 → 7905.97 €
60GEC3700003 → 6200.31 €
60EEC3700005 → 6200.31 €
60FEC3700005 → 6200.31 €
28FEC3000028 → 4470.62 €
28GEC3000046 → 4470.62 €
28FEC3000025 → 4470.62 €
TD-D5TG-105693 → 1113.90 €
TD-E5TG-105643 → 1113.90 €
TD-F5TG-105596 → 1113.90 €
TD-G5TG-105581 → 1113.90 €
60EEC3600003 → 616.76 €
60EEC3000044 → 326.12 €
28FEC3000026 → 219.03 €
60GEC3600003 → 154.19 €
60FEC3600003 → 154.19 €
60FEC3500003 → 154.19 €
60GEC3500003 → 154.19 €

TOTAL: 259206.63 €


PULP CON HASTA 5 SOLUCIONES

In [ ]:
from pulp import LpProblem, LpVariable, LpBinary, lpSum, LpStatus

# Datos
raw_data = [
   ("60GEC3700004", "133.727,53 €"),
   ("60EEC3000022", "125.165,62 €"),
   ("60GEC3000039", "125.165,62 €"),
   ("60FEC3000039", "125.165,62 €"),
   ("60EEC3000023", "125.165,62 €"),
   ("60EEC3000039", "61.636,53 €"),
   ("60GEC3000040", "61.636,53 €"),
   ("60FEC3000040", "61.636,53 €"),
   ("60EEC3000047", "61.636,53 €"),
   ("60EEC3000025", "35.710,05 €"),
   ("60EEC3000026", "35.710,05 €"),
   ("60EEC3000046", "35.710,05 €"),
   ("60EEC3000048", "35.710,05 €"),
   ("60GEC3000041", "35.710,05 €"),
   ("60GEC3000042", "35.710,05 €"),
   ("60FEC3000041", "35.710,05 €"),
   ("60FEC3000042", "35.710,05 €"),
   ("28FEC3000027", "34.790,89 €"),
   ("60EEC3000045", "34.568,64 €"),
   ("60GEC3000043", "34.568,64 €"),
   ("60FEC3000043", "34.568,64 €"),
   ("60EECR000001", "34.405,58 €"),
   ("60EEC3000024", "34.405,58 €"),
   ("35FEC3700001", "7.905,97 €"),
   ("35GEC3700002", "7.905,97 €"),
   ("60EEC3700004", "6.200,31 €"),
   ("60GEC3700003", "6.200,31 €"),
   ("60EEC3700005", "6.200,31 €"),
   ("60FEC3700005", "6.200,31 €"),
   ("28FEC3000028", "4.470,62 €"),
   ("28GEC3000044", "4.470,62 €"),
   ("28GEC3000046", "4.470,62 €"),
   ("28GEC3000047", "4.470,62 €"),
   ("28GEC3000045", "4.470,62 €"),
   ("28FEC3000029", "4.470,62 €"),
   ("28FEC3000025", "4.470,62 €"),
   ("60GEC3600004", "2.760,16 €"),
   ("60GEC3500004", "2.063,36 €"),
   ("TD-D5TG-105693", "1.113,90 €"),
   ("TD-E5TG-105643", "1.113,90 €"),
   ("TD-F5TG-105596", "1.113,90 €"),
   ("TD-G5TG-105581", "1.113,90 €"),
   ("28GEC3000042", "620,00 €"),
   ("60EEC3500003", "616,76 €"),
   ("60EEC3600003", "616,76 €"),
   ("60EEC3000044", "326,12 €"),
   ("28FEC3000026", "219,03 €"),
   ("60EEC3500004", "154,19 €"),
   ("60GEC3600003", "154,19 €"),
   ("60EEC3600004", "154,19 €"),
   ("60FEC3600003", "154,19 €"),
   ("60FEC3500003", "154,19 €"),
   ("60GEC3500003", "154,19 €")
]

# Convertir importes a float
data = [(id, float(imp.replace('.', '').replace(',', '.').replace(' €', ''))) for id, imp in raw_data]

target = 259206.63
max_solutions = 5

# Variables globales para el modelo
vars = {f_id: LpVariable(f"incluye_{f_id}", cat=LpBinary) for f_id, _ in data}

# Función para crear un nuevo modelo con la restricción extra de exclusión
def crear_modelo(excluir_sols):
    model = LpProblem("Seleccion_de_facturas", sense=1)
    model += lpSum(vars[f_id] * importe for f_id, importe in data) == target
    # Añadir restricciones para excluir soluciones previas
    for sol in excluir_sols:
        model += lpSum(vars[f_id] for f_id in sol) <= len(sol) - 1
    return model

soluciones = []
for i in range(max_solutions):
    model = crear_modelo(soluciones)
    status = model.solve()
    if LpStatus[status] != "Optimal":
        print(f"No se encontraron más soluciones después de {i} iteraciones.")
        break

    seleccion = [f_id for f_id, _ in data if vars[f_id].value() == 1]
    print(f"\nSolución {i + 1}:")
    total = 0
    for f_id in seleccion:
        imp = next(imp for id_, imp in data if id_ == f_id)
        print(f"{f_id} → {imp:.2f} €")
        total += imp
    print(f"Total: {total:.2f} €")
    soluciones.append(seleccion)



Solución 1:
60GEC3700004 → 133727.53 €
60EEC3000046 → 35710.05 €
60FEC3000041 → 35710.05 €
35FEC3700001 → 7905.97 €
35GEC3700002 → 7905.97 €
60GEC3700003 → 6200.31 €
60EEC3700005 → 6200.31 €
60FEC3700005 → 6200.31 €
28FEC3000028 → 4470.62 €
28GEC3000046 → 4470.62 €
28FEC3000025 → 4470.62 €
TD-D5TG-105693 → 1113.90 €
TD-E5TG-105643 → 1113.90 €
TD-F5TG-105596 → 1113.90 €
TD-G5TG-105581 → 1113.90 €
60EEC3600003 → 616.76 €
60EEC3000044 → 326.12 €
28FEC3000026 → 219.03 €
60GEC3600003 → 154.19 €
60FEC3600003 → 154.19 €
60FEC3500003 → 154.19 €
60GEC3500003 → 154.19 €
Total: 259206.63 €

Solución 2:
60GEC3700004 → 133727.53 €
60EEC3000046 → 35710.05 €
60FEC3000041 → 35710.05 €
35FEC3700001 → 7905.97 €
35GEC3700002 → 7905.97 €
60EEC3700004 → 6200.31 €
60GEC3700003 → 6200.31 €
60EEC3700005 → 6200.31 €
28FEC3000028 → 4470.62 €
28GEC3000047 → 4470.62 €
28FEC3000025 → 4470.62 €
TD-D5TG-105693 → 1113.90 €
TD-E5TG-105643 → 1113.90 €
TD-F5TG-105596 → 1113.90 €
TD-G5TG-105581 → 1113.90 €
60EEC3600003 

KeyboardInterrupt: 

VERSION PULP CON PESOS

In [ ]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpBinary

# --- Subir archivo Excel ---
from google.colab import files
uploaded = files.upload()
df = pd.read_excel(list(uploaded.keys())[0])

# --- Detectar columna de fecha emisión ---
col_fecha_emision = next((col for col in ['FECHA', 'Fecha', 'fecha', 'Fecha Emision', 'FECHA_EMISION'] if col in df.columns), None)
if not col_fecha_emision:
    raise ValueError("No se encontró la columna de fecha emisión.")

# --- Limpieza de columnas ---
df[col_fecha_emision] = pd.to_datetime(df[col_fecha_emision], dayfirst=True, errors='coerce')
df['IMPORTE'] = df['IMPORTE'].apply(lambda x: float(str(x).replace('.', '').replace(',', '.')))
df['FACTURA'] = df['FACTURA'].astype(str)

# --- Calcular antigüedad ---
fecha_base = df[col_fecha_emision].min()
df['DAYS_FROM_BASE'] = (df[col_fecha_emision] - fecha_base).dt.days

# --- Inputs del usuario ---
importe_input = input("Introduce el importe objetivo (ej: 295.206,63): ").replace('.', '').replace(',', '.')
importe_objetivo_eur = float(importe_input)
importe_objetivo_cent = int(round(importe_objetivo_eur * 100))

fecha_pago_input = input("Introduce la fecha del pago (ej: 28/07/2025): ")
fecha_pago = pd.to_datetime(fecha_pago_input, dayfirst=True, errors='coerce')
if pd.isna(fecha_pago):
    raise ValueError("Fecha de pago no válida.")
print(f"📅 Fecha de pago interpretada: {fecha_pago.date()}")

# --- Convertir importes a céntimos (enteros) ---
df['IMPORTE_CENT'] = (df['IMPORTE'] * 100).round().astype(int)

# --- Función exacta sin límite max_fact ---
def seleccionar_facturas_exactas(df, objetivo_cent, max_fact=None):
    df_filt = df.copy()
    df_filt = df_filt.sort_values(by=['IMPORTE_CENT', 'DAYS_FROM_BASE'], ascending=[False, True])
    if max_fact:
        df_filt = df_filt.head(max_fact)

    data = list(zip(df_filt['FACTURA'], df_filt['IMPORTE_CENT'], df_filt['DAYS_FROM_BASE'], df_filt['IMPORTE']))
    model = LpProblem("Seleccion_de_facturas_exactas", sense=1)

    vars = {f: LpVariable(f"sel_{f}", cat=LpBinary) for f, _, _, _ in data}

    # Restricción suma exacta en céntimos
    model += lpSum(vars[f] * imp_cent for f, imp_cent, _, _ in data) == objetivo_cent

    # Función objetivo: minimizar número de facturas + antigüedad (ligera)
    model += lpSum(vars[f] for f, _, _, _ in data) + 1e-6 * lpSum(vars[f] * days for f, _, days, _ in data)

    if model.solve() == 1:
        seleccionadas = [(f, imp_eur) for f, _, _, imp_eur in data if vars[f].value() == 1]
        total = sum(imp for _, imp in seleccionadas)
        return seleccionadas, total
    return None, None

# --- Ejecutar búsqueda sin límite ---
print(f"\n⏳ Buscando combinación EXACTA para: {importe_objetivo_eur:,.2f} €...")
seleccion, total = seleccionar_facturas_exactas(df, importe_objetivo_cent, max_fact=None)

# --- Mostrar resultados ---
if seleccion:
    print("\n✅ Facturas seleccionadas:")
    for f, imp in seleccion:
        print(f"{f}: {imp:,.2f} €")
    print(f"\n🔢 TOTAL SELECCIONADO: {total:,.2f} €")
else:
    print("❌ No se encontró una combinación EXACTA con las facturas disponibles.")


Saving DetalleDocumentos.xlsx to DetalleDocumentos (4).xlsx
Introduce el importe objetivo (ej: 295.206,63): 295.206,63
Introduce la fecha del pago (ej: 28/07/2025): 28/07/2025
📅 Fecha de pago interpretada: 2025-07-28

⏳ Buscando combinación EXACTA para: 295,206.63 €...
❌ No se encontró una combinación EXACTA con las facturas disponibles.


ORTOOLS

In [ ]:


# --- Subir archivo Excel ---
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# --- Leer Excel ---
df = pd.read_excel(filename, engine="openpyxl")

# --- Detectar columna fecha emisión ---
possible_date_cols = ['FECHA', 'Fecha', 'fecha', 'Fecha Emision', 'FECHA_EMISION']
col_fecha_emision = next((col for col in possible_date_cols if col in df.columns), None)
if not col_fecha_emision:
    raise ValueError("No se encontró columna fecha emisión.")

# --- Detectar columnas factura e importe ---
possible_factura_cols = ['FACTURA', 'Factura', 'factura', 'Nº Factura', 'NRO_FACTURA']
possible_importe_cols = ['IMPORTE', 'Importe', 'importe', 'TOTAL', 'TOTAL_FACTURA']
col_factura = next((col for col in possible_factura_cols if col in df.columns), None)
col_importe = next((col for col in possible_importe_cols if col in df.columns), None)

if not col_factura or not col_importe:
    raise ValueError("No se encontró columna factura o importe.")

# --- Procesar columnas ---
df[col_fecha_emision] = pd.to_datetime(df[col_fecha_emision], dayfirst=True, errors='coerce')
df[col_factura] = df[col_factura].astype(str)

# --- Función para convertir importes con formato europeo ---
def convertir_importe_europeo(valor):
    if pd.isna(valor):
        return None
    if isinstance(valor, (int, float)):
        return float(valor)
    texto = str(valor).strip()
    texto = texto.replace('.', '')  # quitar separadores de miles
    texto = texto.replace(',', '.') # coma decimal a punto decimal
    try:
        return float(texto)
    except:
        return None

# --- Aplicar la función a la columna importe ---
df['IMPORTE_CORRECTO'] = df[col_importe].apply(convertir_importe_europeo)

# Verificar suma total correcta (formato europeo en print)
total = df['IMPORTE_CORRECTO'].sum()
minimo = df['IMPORTE_CORRECTO'].min()
maximo = df['IMPORTE_CORRECTO'].max()
print(f"Número total de facturas: {len(df)}")
print("Suma total importes (convertidos):", f"{total:,.2f} €".replace(",", "X").replace(".", ",").replace("X", "."))
print("Importe mínimo:", f"{minimo:,.2f} €".replace(",", "X").replace(".", ",").replace("X", "."))
print("Importe máximo:", f"{maximo:,.2f} €".replace(",", "X").replace(".", ",").replace("X", "."))

# Calcular antigüedad en días desde fecha base
fecha_base = df[col_fecha_emision].min()
df['DAYS_FROM_BASE'] = (df[col_fecha_emision] - fecha_base).dt.days.fillna(0).astype(int)

# Convertir importe a céntimos enteros para evitar errores de float
df['IMPORTE_CENT'] = (df['IMPORTE_CORRECTO'] * 100).round().astype(int)

# --- Pedir input ---
importe_input = input("Introduce importe objetivo (ej: 295.206,63): ").replace('.', '').replace(',', '.')
importe_objetivo_eur = float(importe_input)
importe_objetivo_cent = int(round(importe_objetivo_eur * 100))

fecha_pago_input = input("Introduce la fecha del pago (ej: 28/07/2025): ")
fecha_pago = pd.to_datetime(fecha_pago_input, dayfirst=True, errors='coerce')
if pd.isna(fecha_pago):
    raise ValueError("Fecha de pago no válida.")
print(f"📅 Fecha de pago interpretada: {fecha_pago.date()}")

# --- ORTOOLS para selección exacta ---
def seleccionar_facturas_exactas_ortools(df, objetivo_cent):
    model = cp_model.CpModel()

    data = list(zip(df[col_factura], df['IMPORTE_CENT'], df['DAYS_FROM_BASE'], df['IMPORTE_CORRECTO']))

    vars = [model.NewBoolVar(f"sel_{f}") for f, _, _, _ in data]

    # Suma exacta igual a objetivo
    model.Add(sum(vars[i] * data[i][1] for i in range(len(data))) == objetivo_cent)

    # Minimizar número de facturas seleccionadas
    model.Minimize(sum(vars))

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10  # Límite de tiempo en segundos
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        seleccionadas = []
        total_cent = 0
        for i, (f, imp_cent, _, imp_eur) in enumerate(data):
            if solver.Value(vars[i]) == 1:
                seleccionadas.append((f, imp_eur))
                total_cent += imp_cent
        return seleccionadas, total_cent / 100
    else:
        return None, None

print(f"\n⏳ Buscando combinación EXACTA para: {importe_objetivo_eur:,.2f} €".replace(",", "X").replace(".", ",").replace("X", "."))

seleccion, total = seleccionar_facturas_exactas_ortools(df, importe_objetivo_cent)

if seleccion:
    print("\n✅ Facturas seleccionadas:")
    for f, imp in seleccion:
        print(f"{f}: {imp:,.2f} €".replace(",", "X").replace(".", ",").replace("X", "."))
    print("\n🔢 TOTAL SELECCIONADO:", f"{total:,.2f} €".replace(",", "X").replace(".", ",").replace("X", "."))
else:
    print("❌ No se encontró una combinación EXACTA de facturas.")


Saving DetalleDocumentos.xlsx to DetalleDocumentos (17).xlsx
Número total de facturas: 54
Suma total importes (convertidos): 4.974.782,07 €
Importe mínimo: 154,19 €
Importe máximo: 3.516.347,12 €
Introduce importe objetivo (ej: 295.206,63): 295.206,63
Introduce la fecha del pago (ej: 28/07/2025): 28/07/2025
📅 Fecha de pago interpretada: 2025-07-28

⏳ Buscando combinación E.ACTA para: 295.206,63 €

✅ Facturas seleccionadas:
60GEC3000040: 61.636,53 €
60EEC3000048: 35.710,05 €
60FEC3000042: 35.710,05 €
60FEC3000041: 35.710,05 €
60GEC3000042: 35.710,05 €
60GEC3000041: 35.710,05 €
60GEC3000043: 34.568,64 €
60EEC3700005: 6.200,31 €
60FEC3700005: 6.200,31 €
60GEC3700003: 6.200,31 €
60EEC3500003: 616,76 €
60EEC3600003: 616,76 €
60FEC3500003: 154,19 €
60FEC3600003: 154,19 €
60GEC3500003: 154,19 €
60GEC3600003: 154,19 €

🔢 TOTAL SELECCIONADO: 295.206,63 €


STREAMLIT

In [30]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.1 MB/s eta 0:00:00


In [32]:
import streamlit as st


st.set_page_config(page_title="Selector de Facturas", layout="wide")
st.title("🧾 Selector de Facturas por Importe Exacto")

# --- Función para convertir formato europeo ---
def convertir_importe_europeo(valor):
    if pd.isna(valor):
        return None
    if isinstance(valor, (int, float)):
        return float(valor)
    texto = str(valor).strip().replace('.', '').replace(',', '.')
    try:
        return float(texto)
    except:
        return None

# --- Subir archivo ---
uploaded_file = st.file_uploader("📂 Sube tu archivo Excel de facturas", type=["xlsx"])
if uploaded_file:
    df = pd.read_excel(uploaded_file, engine="openpyxl")

    # Detectar columnas
    fecha_cols = ['FECHA', 'Fecha', 'fecha', 'Fecha Emision', 'FECHA_EMISION']
    venc_cols = ['VENCIMIENTO', 'Fecha Vto', 'Vto', 'Fecha Vencimiento', 'FECHA_VTO']
    factura_cols = ['FACTURA', 'Factura', 'factura', 'Nº Factura', 'NRO_FACTURA']
    importe_cols = ['IMPORTE', 'Importe', 'importe', 'TOTAL', 'TOTAL_FACTURA']

    col_fecha = next((col for col in fecha_cols if col in df.columns), None)
    col_vto = next((col for col in venc_cols if col in df.columns), None)
    col_factura = next((col for col in factura_cols if col in df.columns), None)
    col_importe = next((col for col in importe_cols if col in df.columns), None)

    if not col_fecha or not col_factura or not col_importe:
        st.error("⚠️ Faltan columnas obligatorias en el Excel.")
    else:
        df[col_fecha] = pd.to_datetime(df[col_fecha], dayfirst=True, errors='coerce')
        if col_vto:
            df[col_vto] = pd.to_datetime(df[col_vto], dayfirst=True, errors='coerce')
        df[col_factura] = df[col_factura].astype(str)
        df['IMPORTE_CORRECTO'] = df[col_importe].apply(convertir_importe_europeo)
        df['IMPORTE_CENT'] = (df['IMPORTE_CORRECTO'] * 100).round().astype(int)

        # Mostrar resumen
        st.subheader("📊 Resumen de Facturas")
        st.write(f"Número total de facturas: **{len(df)}**")
        st.write(f"Suma total importes: **{df['IMPORTE_CORRECTO'].sum():,.2f} €**")

        # Inputs usuario
        col1, col2 = st.columns(2)
        with col1:
            importe_input = st.text_input("🎯 Importe objetivo (ej: 295.206,63)", "")
        with col2:
            fecha_pago_input = st.date_input("📅 Fecha del pago")

        if importe_input and fecha_pago_input:
            try:
                importe_float = float(importe_input.replace('.', '').replace(',', '.'))
                importe_cent = int(round(importe_float * 100))
            except:
                st.error("❌ Formato de importe no válido.")
                st.stop()

            # OR-Tools para encontrar combinación exacta
            def buscar_combinacion(df, objetivo_cent):
                model = cp_model.CpModel()
                data = list(zip(
                    df[col_factura],
                    df['IMPORTE_CENT'],
                    df[col_fecha],
                    df[col_vto] if col_vto else [None] * len(df),
                    df['IMPORTE_CORRECTO']
                ))

                vars = [model.NewBoolVar(f"sel_{i}") for i in range(len(data))]
                model.Add(sum(vars[i] * data[i][1] for i in range(len(data))) == objetivo_cent)
                model.Minimize(sum(vars))

                solver = cp_model.CpSolver()
                status = solver.Solve(model)

                if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
                    resultado = []
                    for i, (fact, cent, fecha, vto, imp) in enumerate(data):
                        if solver.Value(vars[i]):
                            resultado.append({
                                "Factura": fact,
                                "Importe (€)": round(imp, 2),
                                "Fecha Emisión": fecha.strftime('%d/%m/%Y') if pd.notna(fecha) else '',
                                "Vencimiento": vto.strftime('%d/%m/%Y') if col_vto and pd.notna(vto) else ''
                            })
                    return resultado
                return None

            st.info(f"🔎 Buscando combinación exacta para {importe_float:,.2f} €...")
            resultado = buscar_combinacion(df, importe_cent)

            if resultado:
                st.success("✅ ¡Combinación encontrada!")
                st.dataframe(pd.DataFrame(resultado))
            else:
                st.error("❌ No se encontró una combinación exacta.")



2025-07-31 12:37:53.016 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-31 12:37:53.019 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-31 12:37:53.020 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-31 12:37:53.021 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-31 12:37:53.023 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-31 12:37:53.023 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-31 12:37:53.024 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-31 12:37:53.026 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar